In [ ]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import fashion_mnist,mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from scipy.stats import pearsonr
from tqdm import tqdm
from scipy import ndimage
from IPython.display import clear_output

Using TensorFlow backend.


In [ ]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 28
NUM_CLASSES = 10
NUM_CHANNELS = 1
MODEL_NAME = "MNIST_rotation"
VOTING = 'HARD'
PATH = "/content/drive/My Drive/NC/"
MODEL_ADDITION_DELTA = 0.01
MODEL_ADDITION_PATIENCE = 3
NR_OF_RUNS = 10

# Preprocess

In [ ]:
def preprocess(imgs):
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [ ]:
(x_train_val, y_train_val), (x_test, y_test) = mnist.load_data()

x_train_val = preprocess(x_train_val)
x_test = preprocess(x_test)

print('x_train shape:', x_train_val.shape)
print(x_train_val.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# Convert class vectors to binary class matrices.
y_train_val = keras.utils.to_categorical(y_train_val, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
x_train_val = x_train_val.astype('float32')
x_test = x_test.astype('float32')
x_train_val /= 255
x_test /= 255

# Model

In [ ]:
def MNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size = (2,2), strides=(2,2), padding = "same")(x)
    x = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters = 10, kernel_size = (1,1),strides = (1,1), padding = 'valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

#Predict


In [ ]:
def hard_voting(models, X):
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(X), axis=1))

    prediction = np.transpose(predictions)
    prediction = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=prediction)

    return prediction

def soft_voting(models, X):
    predictions = np.empty((len(X),0,NUM_CLASSES))

    for m in models:
        pred = np.expand_dims(m.predict(X), axis=1)
        predictions = np.append(predictions, pred, axis=1)

    predictions = np.apply_along_axis(np.transpose, axis=1, arr=predictions)
    predictions = np.mean(predictions, axis=1)
    prediction = np.argmax(predictions, axis=1)

    return prediction

def predict(models, X, Y,voting = 'hard'):
    
    if voting == "soft":
      prediction = soft_voting(models, X)
    elif voting == "hard":
      prediction = hard_voting(models, X)
    else:
      raise ValueError(f"Voting mechanism: {VOTING} not supported")

    return accuracy_score(prediction, np.argmax(Y, axis=1))

#Augmentation 


In [ ]:
def rotate_image(image):
    if np.random.rand() < 0.2:
        angles = np.linspace(1,10,10)
        rotation_angle = np.random.choice(angles)
        if np.random.rand() < 0.5:
            image = ndimage.rotate(image, rotation_angle, reshape = False)
        else:
            image = ndimage.rotate(image, -rotation_angle, reshape = False) 
    return image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Train

In [ ]:
for run in range(1, NR_OF_RUNS+1):

    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.20, shuffle= True)

    models = []
    accuracies = [0]
    patience = 0

    for i in count(1):

        print(f"\n ===== Train model {i} =====")

        # Set the seeds
        np.random.seed(run*i)
        tf.random.set_seed(run*i)

        # augmentation
        datagen = ImageDataGenerator(preprocessing_function=rotate_image)
        datagen = datagen.flow(x_train,y_train, batch_size= BATCH_SIZE)

        # Create directories
        os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
        os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)

        # Create the model
        model = MNISTmodel(IMAGE_SIZE, NUM_CLASSES, 1)
        
        # Load the weighs if the model is already trained
        weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-{i}.h5"

        if os.path.exists(weights_path):
            print(f"Skipping training of model {i}: weights exists")
            model.load_weights(weights_path)
        else:
            es = EarlyStopping(min_delta=0.01, patience=3)
            csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history-{i}.csv", separator=';')

            model.fit_generator(datagen,
                                epochs = EPOCHS,
                                validation_data = (x_val, y_val),
                                shuffle = True,
                                callbacks=[es, csv_logger])
            
            model.save_weights(weights_path)
        
        models.append(model)

        acc = predict(models, x_val, y_val)
        delta = acc - accuracies[-1]

        accuracies.append(acc)

        if delta >= MODEL_ADDITION_DELTA:
          patience = 0
        else:
          patience += 1

        print(f"Model: {i} added. Resulting score: {acc}, Delta: {delta}, Patience: {patience}")

        if patience >= MODEL_ADDITION_PATIENCE:
          break

    # Results

    ## Accuracy vs nr of models
    ## Visualizing the accuracy vs the number of models in the ensamble

    print("\n ===== Accuracy vs nr of models =====")

    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df.insert(1, "Nr of models", accuracy_df.index)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy_SOFT.csv")
    display(accuracy_df)

    ## Accuracy
    ## The final accuracy of the ensamble on the test set
    print("\n ===== Accuracy ======")

    accuracy = predict(models, x_test, y_testc,voting='soft')
    print("Accuracy: " + str(accuracy))

    ## Correlation between models
    print("\n ===== Correlation =====")
    predictions = []

    for m in models:
        predictions.append(np.argmax(m.predict(x_test), axis=1))
    classified = []

    for prediction in predictions:
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
    correlation_matrix = []

    for ix, x in enumerate(classified):
      row = []
      
      for iy, y in enumerate(classified):
        if (ix == iy):
          row.append(np.nan)
        else:
          row.append(pearsonr(x,y)[0])

      correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix_SOFT.csv")
    
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))

    # Save the results
    file = PATH + MODEL_NAME + f"/results_SOFT.csv"
    df = pd.DataFrame([[run, accuracy, correlation]])

    if not os.path.isfile(file):
      df.to_csv(file, header=["run", "accuracy", "correlation"], index=False)
    else: # else it exists so append without writing the header
      df.to_csv(file, mode='a', header=False, index=False)



 ===== Train model 1 =====
Skipping training of model 1: weights exists
Model: 1 added. Resulting score: 0.7716666666666666, Delta: 0.7716666666666666, Patience: 0

 ===== Train model 2 =====
Skipping training of model 2: weights exists
Model: 2 added. Resulting score: 0.76725, Delta: -0.004416666666666624, Patience: 1

 ===== Train model 3 =====
Skipping training of model 3: weights exists
Model: 3 added. Resulting score: 0.7781666666666667, Delta: 0.010916666666666686, Patience: 0

 ===== Train model 4 =====
Skipping training of model 4: weights exists
Model: 4 added. Resulting score: 0.7645, Delta: -0.013666666666666716, Patience: 1

 ===== Train model 5 =====
Skipping training of model 5: weights exists
Model: 5 added. Resulting score: 0.7669166666666667, Delta: 0.0024166666666667336, Patience: 2

 ===== Train model 6 =====
Skipping training of model 6: weights exists
Model: 6 added. Resulting score: 0.76225, Delta: -0.004666666666666708, Patience: 3

 ===== Accuracy vs nr of mode

,Accuracy,Nr of models
0,0.000000,0
1,0.771667,1
2,0.767250,2
3,0.778167,3
4,0.764500,4
5,0.766917,5
6,0.762250,6



 ===== Accuracy ======
Accuracy: 0.7872

 ===== Correlation =====


,0,1,2,3,4,5
0,NaN,0.636596,0.621804,0.612287,0.580650,0.613295
1,0.636596,NaN,0.638040,0.631204,0.616371,0.634965
2,0.621804,0.638040,NaN,0.648346,0.638851,0.625091
3,0.612287,0.631204,0.648346,NaN,0.637326,0.654753
4,0.580650,0.616371,0.638851,0.637326,NaN,0.658641
5,0.613295,0.634965,0.625091,0.654753,0.658641,NaN


Average correlation: 0.6298813553234137

 ===== Train model 1 =====
Skipping training of model 1: weights exists
Model: 1 added. Resulting score: 0.7621666666666667, Delta: 0.7621666666666667, Patience: 0

 ===== Train model 2 =====
Skipping training of model 2: weights exists
Model: 2 added. Resulting score: 0.7406666666666667, Delta: -0.021499999999999964, Patience: 1

 ===== Train model 3 =====
Skipping training of model 3: weights exists
Model: 3 added. Resulting score: 0.76475, Delta: 0.024083333333333345, Patience: 0

 ===== Train model 4 =====
Skipping training of model 4: weights exists
Model: 4 added. Resulting score: 0.7565833333333334, Delta: -0.008166666666666655, Patience: 1

 ===== Train model 5 =====
Skipping training of model 5: weights exists
Model: 5 added. Resulting score: 0.7649166666666667, Delta: 0.008333333333333304, Patience: 2

 ===== Train model 6 =====
Skipping training of model 6: weights exists
Model: 6 added. Resulting score: 0.76625, Delta: 0.001333333333

...

Average correlation: 0.6312182343420885

 ===== Train model 1 =====
Skipping training of model 1: weights exists
Model: 1 added. Resulting score: 0.7515833333333334, Delta: 0.7515833333333334, Patience: 0

 ===== Train model 2 =====
Skipping training of model 2: weights exists
Model: 2 added. Resulting score: 0.7369166666666667, Delta: -0.014666666666666717, Patience: 1

 ===== Train model 3 =====
Skipping training of model 3: weights exists
Model: 3 added. Resulting score: 0.7695833333333333, Delta: 0.03266666666666662, Patience: 0

 ===== Train model 4 =====
Skipping training of model 4: weights exists
Model: 4 added. Resulting score: 0.7643333333333333, Delta: -0.005249999999999977, Patience: 1

 ===== Train model 5 =====
Skipping training of model 5: weights exists
Model: 5 added. Resulting score: 0.7716666666666666, Delta: 0.007333333333333303, Patience: 2

 ===== Train model 6 =====
Skipping training of model 6: weights exists
Model: 6 added. Resulting score: 0.77325, Delta: 0.00

,Accuracy,Nr of models
0,0.000000,0
1,0.751583,1
2,0.736917,2
3,0.769583,3
4,0.764333,4
5,0.771667,5
6,0.773250,6



 ===== Accuracy ======
Accuracy: 0.7806

 ===== Correlation =====


,0,1,2,3,4,5
0,NaN,0.659867,0.612157,0.639870,0.639395,0.610384
1,0.659867,NaN,0.615612,0.650013,0.645870,0.602054
2,0.612157,0.615612,NaN,0.647393,0.627546,0.598234
3,0.639870,0.650013,0.647393,NaN,0.655338,0.637430
4,0.639395,0.645870,0.627546,0.655338,NaN,0.672512
5,0.610384,0.602054,0.598234,0.637430,0.672512,NaN


Average correlation: 0.6342448715671699


# Results

In [ ]:
from scipy import stats

In [ ]:
baseline_hard =  [0.7823,
0.7817,
0.7804,
0.7856,
0.7864,
0.7904,
0.7897,
0.7813,
0.7819,
0.784]

In [ ]:
np.mean(baseline_hard)

0.78437

In [ ]:
baseline_soft=   [0.7909,
0.7838,
0.7877,
0.7931,
0.7886,
0.7895,
0.7923,
0.7826,
0.7891,
0.7882]

In [ ]:
np.mean(baseline_soft)

0.7885800000000001

In [ ]:
augmented_hard = pd.read_csv(PATH+ "MNIST_rotation/results_HARD.csv")
augmented_hard = augmented_hard.values
augmented_hard

array([[ 1.        ,  0.7791    ,  0.62988136],
       [ 2.        ,  0.7837    ,  0.63305863],
       [ 3.        ,  0.7794    ,  0.64522595],
       [ 4.        ,  0.7829    ,  0.63789933],
       [ 5.        ,  0.7814    ,  0.62154537],
       [ 6.        ,  0.7786    ,  0.64296796],
       [ 7.        ,  0.7746    ,  0.63130453],
       [ 8.        ,  0.7778    ,  0.63993   ],
       [ 9.        ,  0.7791    ,  0.63121823],
       [10.        ,  0.7728    ,  0.63424487]])

In [ ]:
augmented_soft = pd.read_csv(PATH+"MNIST_rotation/results_SOFT.csv")
augmented_soft = augmented_soft.values
augmented_soft

array([[ 1.        ,  0.7791    ,  0.62988136],
       [ 2.        ,  0.7837    ,  0.63305863],
       [ 3.        ,  0.7794    ,  0.64522595],
       [ 4.        ,  0.7829    ,  0.63789933],
       [ 5.        ,  0.7814    ,  0.62154537],
       [ 6.        ,  0.7786    ,  0.64296796],
       [ 7.        ,  0.7746    ,  0.63130453],
       [ 8.        ,  0.7778    ,  0.63993   ],
       [ 9.        ,  0.7791    ,  0.63121823],
       [10.        ,  0.7728    ,  0.63424487],
       [ 1.        ,  0.7872    ,  0.62988136],
       [ 2.        ,  0.7869    ,  0.63305863],
       [ 3.        ,  0.7835    ,  0.64522595],
       [ 4.        ,  0.7875    ,  0.63789933],
       [ 5.        ,  0.7882    ,  0.62154537],
       [ 6.        ,  0.786     ,  0.64296796],
       [ 7.        ,  0.7842    ,  0.63130453],
       [ 8.        ,  0.783     ,  0.63993   ],
       [ 9.        ,  0.7878    ,  0.63121823],
       [10.        ,  0.7806    ,  0.63424487]])

In [ ]:
augmented_hard = augmented_hard[:,1]
augmented_soft = augmented_soft[:,1][10:]

In [ ]:
np.mean(augmented_hard)

0.77894

In [ ]:
np.mean(augmented_soft)

0.78549

In [ ]:
stats.ttest_rel(augmented_hard,baseline_hard)

Ttest_relResult(statistic=-3.161700451701639, pvalue=0.011518672925021027)

In [ ]:
stats.ttest_rel(augmented_soft,baseline_soft)

Ttest_relResult(statistic=-2.7401461862275154, pvalue=0.022842109845735058)

## Accuracy
The final accuracy of the ensamble on the test set

In [ ]:
!zip -r /content/FashionMNIST_rotation.zip /content/FashionMNIST_rotation

  adding: content/FashionMNIST_rotation/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/accuracy_SOFT.csv (deflated 38%)
  adding: content/FashionMNIST_rotation/10/weights/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/weights/weights-1.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-5.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-2.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-6.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-4.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/weights/weights-3.h5 (deflated 71%)
  adding: content/FashionMNIST_rotation/10/history/ (stored 0%)
  adding: content/FashionMNIST_rotation/10/history/history-4.csv (deflated 52%)
  adding: content/FashionMNIST_rotation/10/history/history-1.csv (deflated 52%)
  adding: content/FashionMNIST_rotation/10/history/hi